# Basic functionalities of asparagus 

This notebook shows the basic functionalities of asparagus. We will use NH3 as an example molecule.

We will test different sampling methods, train of a model and test it.

## Sampling

### Molecular dynamics



In [ ]:
import os
import numpy as np

from ase import io
from ase import units
from ase.optimize import BFGS

from asparagus import MDSampler
    
sampler = MDSampler(
    config='nh3_config_md.json',
    sample_directory='sampling_nh3_md',
    sample_data_file='sampling_nh3_md/md_nh3.db',
    sample_systems=['data/nh3_c3v.xyz'],
    sample_systems_format='xyz',
    sample_systems_optimize=True,
    sample_systems_optimize_fmax=0.001,
    md_temperature=500,
    md_time_step=1.0,
    md_simulation_time=10_000.0,
    md_save_interval=10,
    md_langevin_friction=0.01,
    md_equilibration_time=0,
    md_initial_velocities=False
    )
sampler.run()

### Normal mode scan

In [ ]:
from asparagus import NormalModeScanner
    
sampler = NormalModeScanner(
    config='nh3_config_nms.json',
    sample_directory='sampling_nh3_nms',
    sample_data_file='sampling_nh3_nms/nms_nh3.db',
    sample_systems=['data/nh3_c3v.xyz'],
    sample_systems_format='xyz',
    sample_systems_optimize=True,
    sample_systems_optimize_fmax=0.001,
    nms_harmonic_energy_step=0.03,
    nms_energy_limits=1.00,
    nms_number_of_coupling=2,
    )
sampler.run(
    nms_exclude_modes=np.arange(6))

### Metadynamics sampling

In [ ]:
from asparagus import MetaSampler

sampler = MetaSampler(
    config='nh3_config_meta.json',
    sample_directory='sampling_nh3_meta',
    sample_data_file='sampling_nh3_meta/meta_nh3.db',
    sample_systems='data/nh3_c3v.xyz',
    sample_systems_format='xyz',
    sample_systems_optimize=True,
    sample_systems_optimize_fmax=0.001,
    meta_cv=[[0, 1], [0, 2], [0, 3]],
    meta_gaussian_height=0.05,
    meta_gaussian_widths=0.1,
    meta_gaussian_interval=10,
    meta_hookean=[[0, 1, 4.0], [0, 2, 4.0], [0, 3, 4.0]],
    meta_temperature=500,
    meta_time_step=1.0,
    meta_simulation_time=10_000.0,
    meta_save_interval=10,
    )
sampler.run()


Now that we have sampled the potential energy surfaces with the different methods, we proceed to train the model.



## Training

We will take the database generated from the metadynamics sampling as an example. In this example, we will train a model for the energy, forces and dipole moment fpr 1000 epochs.

In [1]:
from asparagus import Asparagus

model = Asparagus(
    config='nh3_config_meta.json',
    data_file='sampling_nh3_meta/meta_nh3.db',
    model_directory='sampling_nh3_meta',
    model_properties=['energy', 'forces', 'dipole'],
    model_interaction_cutoff=8.0,
    trainer_properties_weights={
        'energy': 1.,
        'forces': 50.,
        'dipole': 25.
        },
    trainer_max_epochs=1_000,
    )
model.train()


/home/vazquez/miniforge3/envs/asparagus/lib/python3.8/site-packages/h5py/__init__.py:36: UserWarning: h5py is running against HDF5 1.14.3 when it was built against 1.14.2, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "
INFO:asparagus.src.settings.config:INFO:
Global parameter configuration update of 'nh3_config_meta.json'.
Current configuration entries will be overwritten in case of conflicting entries.

INFO:asparagus.src.settings.config:INFO:
Conflict! Overwrite parameter 'data_file'.
Conflict! Overwrite parameter 'model_directory'.
Conflict! Overwrite parameter 'model_properties'.
Conflict! Overwrite parameter 'model_interaction_cutoff'.
Conflict! Overwrite parameter 'trainer_properties_weights'.
Conflict! Overwrite parameter 'trainer_max_epochs'.

INFO:asparagus.src.settings.config:INFO:
Global parameter configuration update of 'nh3_config_meta.json'.
Current configuration entries will be overwritten in case of conflicting entrie


       '   _______                                                  ______                  _ _        
       '  (_______)                                                (____  \                | | |       
       '   _______  ___ ____  _____  ____ _____  ____ _   _  ___    ____)  )_   _ ____   __| | | _____ 
       '  |  ___  |/___)  _ \(____ |/ ___|____ |/ _  | | | |/___)  |  __  (| | | |  _ \ / _  | || ___ |
       '  | |   | |___ | |_| / ___ | |   / ___ ( (_| | |_| |___ |  | |__)  ) |_| | | | ( (_| | || ____|
       '  |_|   |_(___/|  __/\_____|_|   \_____|\___ |____/(___/   |______/|____/|_| |_|\____|\_)_____)
       '               |_|                     (_____|                            
       '
       '                        Authors: K. Toepfer and L.I. Vazquez-Salazar
       '                        Date: 2023-12-20 14:18:30
       '                        Task: Training a Neural Network Potential               
       '                        Running on: pc-mm016 with: 

INFO:asparagus.src.settings.config:INFO:
Global parameter configuration update of 'nh3_config_meta.json'.
Current configuration entries will be overwritten in case of conflicting entries.

INFO:asparagus.src.settings.config:INFO:
Global parameter configuration update of 'nh3_config_meta.json'.
Current configuration entries will be overwritten in case of conflicting entries.

INFO:asparagus.src.settings.config:INFO:
Conflict! Overwrite parameter 'data_container'.
Conflict! Overwrite parameter 'test_datasets'.
Conflict! Overwrite parameter 'test_properties'.
Conflict! Overwrite parameter 'test_store_neighbor_list'.



Epoch   999 |##########################################| 100.0% Done - Epoch Time:  2.4 s, Loss:  0.0431   


INFO:asparagus.src.train.tester:INFO:
Summary for train set:
  Property Metrics    MAE,       RMSE
   energy           3.86e-02,  3.91e-02 eV
   forces           2.71e-02,  4.02e-02 eV/Ang
   dipole           1.71e-02,  2.19e-02 eAng

INFO:asparagus.src.train.tester:INFO:
Summary for valid set:
  Property Metrics    MAE,       RMSE
   energy           3.92e-02,  3.97e-02 eV
   forces           3.01e-02,  5.59e-02 eV/Ang
   dipole           1.83e-02,  2.40e-02 eAng

INFO:asparagus.src.train.tester:INFO:
Summary for test set:
  Property Metrics    MAE,       RMSE
   energy           3.90e-02,  3.93e-02 eV
   forces           2.95e-02,  4.51e-02 eV/Ang
   dipole           1.63e-02,  2.11e-02 eAng



## Testing

Once the training is done, we can test the model on the test set. This generates several plots in the model directory.
at the end also prints a summary for the MAE and RMSE for the different properties.

In [2]:
model.test(
    test_datasets='all',
    test_directory=model.get('model_directory'))

INFO:asparagus.src.settings.config:INFO:
Global parameter configuration update of 'nh3_config_meta.json'.
Current configuration entries will be overwritten in case of conflicting entries.

INFO:asparagus.src.settings.config:INFO:
Global parameter configuration update of 'nh3_config_meta.json'.
Current configuration entries will be overwritten in case of conflicting entries.

INFO:asparagus.src.settings.config:INFO:
Conflict! Overwrite parameter 'data_container'.
Conflict! Overwrite parameter 'test_datasets'.
Conflict! Overwrite parameter 'test_properties'.
Conflict! Overwrite parameter 'test_store_neighbor_list'.

INFO:asparagus.src.train.tester:INFO:
Summary for train set:
  Property Metrics    MAE,       RMSE
   energy           3.86e-02,  3.91e-02 eV
   forces           2.71e-02,  4.02e-02 eV/Ang
   dipole           1.71e-02,  2.19e-02 eAng

INFO:asparagus.src.train.tester:INFO:
Summary for valid set:
  Property Metrics    MAE,       RMSE
   energy           3.92e-02,  3.97e-02 eV
 